In [2]:
from dotenv import load_dotenv
import os

In [25]:
load_dotenv()

True

In [27]:
assert(os.getenv('hf_token'))
assert(os.getenv('WANDB_API_KEY'))

In [18]:
%pip install peft wandb

   ---------------------------------------- 0.0/20.7 MB ? eta -:--:--
   - -------------------------------------- 0.5/20.7 MB 3.3 MB/s eta 0:00:07
   ---- ----------------------------------- 2.1/20.7 MB 6.2 MB/s eta 0:00:04
   -------- ------------------------------- 4.2/20.7 MB 7.4 MB/s eta 0:00:03
   ----------- ---------------------------- 5.8/20.7 MB 7.5 MB/s eta 0:00:02
   --------------- ------------------------ 8.1/20.7 MB 8.2 MB/s eta 0:00:02
   -------------------- ------------------- 10.5/20.7 MB 8.7 MB/s eta 0:00:02
   ------------------------ --------------- 12.8/20.7 MB 9.0 MB/s eta 0:00:01
   ----------------------------- ---------- 15.5/20.7 MB 9.5 MB/s eta 0:00:01
   ---------------------------------- ----- 18.1/20.7 MB 9.8 MB/s eta 0:00:01
   ---------------------------------------  20.7/20.7 MB 10.1 MB/s eta 0:00:01
   ---------------------------------------  20.7/20.7 MB 10.1 MB/s eta 0:00:01
   ---------------------------------------  20.7/20.7 MB 10.1 MB/s eta 0:00


[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import PrefixTuningConfig, get_peft_model
import wandb
from huggingface_hub import login
import os

In [22]:
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer

In [28]:
wandb.login(key=os.getenv('WANDB_API_KEY'))
wandb.init(project="prefix-tuning-qwen-25-question-generation", name='qwen25-0.5b-prefix-tuning')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Ravindu\_netrc
wandb: Currently logged in as: rtweera (rtw-rtweera) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [29]:
login(token=os.getenv('hf_token'))

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [30]:
dataset = load_dataset('squad')

README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

C:\Users\Ravindu\Documents\My Projects\slm-fine-tune\venv-slm-fine-tune\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Ravindu\.cache\huggingface\hub\datasets--squad. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [31]:
import pandas as pd

In [32]:
type(dataset)

datasets.dataset_dict.DatasetDict

In [57]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [39]:
from pprint import pprint

In [46]:
pprint(dataset['train'][234])

{'answers': {'answer_start': [74],
             'text': ['Michigan Wolverines football team']},
 'context': 'The Notre Dame football team has a long history, first beginning '
            'when the Michigan Wolverines football team brought football to '
            'Notre Dame in 1887 and played against a group of students. In the '
            'long history since then, 13 Fighting Irish teams have won '
            'consensus national championships (although the university only '
            'claims 11), along with another nine teams being named national '
            'champion by at least one source. Additionally, the program has '
            'the most members in the College Football Hall of Fame, is tied '
            'with Ohio State University with the most Heisman Trophies won, '
            'and have the highest winning percentage in NCAA history. With the '
            'long history, Notre Dame has accumulated many rivals, and its '
            'annual game against USC for the

In [35]:
model_name = 'Qwen/Qwen2.5-0.5B'
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

C:\Users\Ravindu\Documents\My Projects\slm-fine-tune\venv-slm-fine-tune\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Ravindu\.cache\huggingface\hub\models--Qwen--Qwen2.5-0.5B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [48]:
from datasets import Dataset, DatasetDict

In [66]:
def preprocess(example_batch):
    prompt = 'Generate a question from this given context:'
    inputs = [f'{prompt} {context} -> {question}' for context, question in zip(example_batch['context'], example_batch['question'])]
    tokenized = tokenizer(
        inputs,
        max_length=256,
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    )
    tokenized['labels'] = tokenized['input_ids'].clone()
    return tokenized
    

In [67]:
dataset.map(lambda x: print(type(x)), batched=True)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

<class 'datasets.formatting.formatting.LazyBatch'>
<class 'datasets.formatting.formatting.LazyBatch'>
<class 'datasets.formatting.formatting.LazyBatch'>
<class 'datasets.formatting.formatting.LazyBatch'>
<class 'datasets.formatting.formatting.LazyBatch'>
<class 'datasets.formatting.formatting.LazyBatch'>
<class 'datasets.formatting.formatting.LazyBatch'>
<class 'datasets.formatting.formatting.LazyBatch'>
<class 'datasets.formatting.formatting.LazyBatch'>
<class 'datasets.formatting.formatting.LazyBatch'>
<class 'datasets.formatting.formatting.LazyBatch'>
<class 'datasets.formatting.formatting.LazyBatch'>
<class 'datasets.formatting.formatting.LazyBatch'>
<class 'datasets.formatting.formatting.LazyBatch'>
<class 'datasets.formatting.formatting.LazyBatch'>
<class 'datasets.formatting.formatting.LazyBatch'>
<class 'datasets.formatting.formatting.LazyBatch'>
<class 'datasets.formatting.formatting.LazyBatch'>
<class 'datasets.formatting.formatting.LazyBatch'>
<class 'datasets.formatting.for

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

<class 'datasets.formatting.formatting.LazyBatch'>
<class 'datasets.formatting.formatting.LazyBatch'>
<class 'datasets.formatting.formatting.LazyBatch'>
<class 'datasets.formatting.formatting.LazyBatch'>
<class 'datasets.formatting.formatting.LazyBatch'>
<class 'datasets.formatting.formatting.LazyBatch'>
<class 'datasets.formatting.formatting.LazyBatch'>
<class 'datasets.formatting.formatting.LazyBatch'>
<class 'datasets.formatting.formatting.LazyBatch'>
<class 'datasets.formatting.formatting.LazyBatch'>
<class 'datasets.formatting.formatting.LazyBatch'>


DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [68]:
tokenized = dataset.map(preprocess, batched=True, remove_columns = dataset['train'].column_names)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [69]:
tokenized

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10570
    })
})

In [72]:
tokenized['validation'][0]

{'input_ids': [31115,
  264,
  3405,
  504,
  419,
  2661,
  2266,
  25,
  7297,
  20288,
  220,
  20,
  15,
  572,
  458,
  3693,
  8964,
  1809,
  311,
  8253,
  279,
  18319,
  315,
  279,
  5055,
  20761,
  8953,
  320,
  87101,
  8,
  369,
  279,
  220,
  17,
  15,
  16,
  20,
  3200,
  13,
  576,
  3693,
  20761,
  14872,
  320,
  32,
  6754,
  8,
  18319,
  22117,
  41594,
  23283,
  279,
  5055,
  20761,
  14872,
  320,
  45,
  6754,
  8,
  18319,
  12740,
  44167,
  220,
  17,
  19,
  4142,
  16,
  15,
  311,
  7232,
  862,
  4843,
  7297,
  20288,
  2265,
  13,
  576,
  1809,
  572,
  6342,
  389,
  7400,
  220,
  22,
  11,
  220,
  17,
  15,
  16,
  21,
  11,
  518,
  55041,
  594,
  22636,
  304,
  279,
  5836,
  12879,
  9154,
  12030,
  518,
  15993,
  50557,
  11,
  7043,
  13,
  1634,
  419,
  572,
  279,
  220,
  20,
  15,
  339,
  7297,
  20288,
  11,
  279,
  10734,
  45628,
  279,
  330,
  97235,
  21582,
  1,
  448,
  5257,
  6623,
  56589,
  27172,
  11,
  438,
  

In [80]:
train_set = tokenized['train'].select(range(1000))
eval_set = tokenized['validation'].select(range(200))

In [75]:
base_model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")

config.json:   0%|          | 0.00/681 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

In [76]:
# Function to generate questions
def generate_question(context, model, tokenizer, max_new_tokens=30):
    input_text = f"Generate a question: {context} -> "
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        max_length=256,
        truncation=True
    ).to(model.device)
    question_ids = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=max_new_tokens,
        num_beams=4,
        early_stopping=True,
        pad_token_id=tokenizer.pad_token_id
    )
    question = tokenizer.decode(question_ids[0], skip_special_tokens=True)
    # Extract question after "->"
    question = question.split("->")[-1].strip()
    return question

In [79]:
# Evaluate base model
scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)
base_bleu_scores = []
base_rouge_scores = []

In [85]:
# Flatten validation dataset for evaluation
eval_contexts = []
eval_questions = []
for example in dataset["validation"]:
    eval_contexts.append(example['context'])
    eval_questions.append(example['question'])

In [86]:
from tqdm import tqdm

In [87]:
# Evaluate on subset
for i in tqdm(range(len(eval_set))):  # Match small_eval size
    context = eval_contexts[i]
    reference = eval_questions[i]
    question = generate_question(context, base_model, tokenizer)
    # BLEU score
    bleu_score = sentence_bleu(reference.split(), question.split())
    base_bleu_scores.append(bleu_score)
    # ROUGE-L score
    rouge_score = scorer.score(reference, question)["rougeL"].fmeasure
    base_rouge_scores.append(rouge_score)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [28:38<00:00,  8.59s/it]


In [88]:
# Compute average scores
base_bleu_avg = np.mean(base_bleu_scores)
base_rouge_avg = np.mean(base_rouge_scores)
print("Base Model Scores:", {"bleu": base_bleu_avg, "rougeL": base_rouge_avg})

# Log to W&B
wandb.log({"base_bleu": base_bleu_avg, "base_rougeL": base_rouge_avg})

Base Model Scores: {'bleu': np.float64(2.185774629528745e-232), 'rougeL': np.float64(0.15532109414983492)}


### Tuning part

In [89]:
# Step 3: Configure Prefix Tuning
# Define prefix tuning configuration
peft_config = PrefixTuningConfig(
    task_type="CAUSAL_LM",
    num_virtual_tokens=20,
    prefix_projection=True
)

In [90]:
# Load model and apply prefix tuning
model = AutoModelForCausalLM.from_pretrained(model_name)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# Move to GPU if available
model = model.to("cuda" if torch.cuda.is_available() else "cpu")

trainable params: 811,648 || all params: 494,844,416 || trainable%: 0.1640


In [91]:
# Step 4: Define Metrics and Training Arguments
# Compute BLEU and ROUGE metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Extract question part after "->"
    decoded_preds = [pred.split("->")[-1].strip() if "->" in pred else pred for pred in decoded_preds]
    decoded_labels = [label.split("->")[-1].strip() if "->" in label else label for label in decoded_labels]
    scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)
    bleu_scores = []
    rouge_scores = []
    for pred, ref in zip(decoded_preds, decoded_labels):
        bleu_scores.append(sentence_bleu([ref.split()], pred.split()))
        rouge_scores.append(scorer.score(ref, pred)["rougeL"].f1)
    return {
        "bleu": np.mean(bleu_scores),
        "rougeL": np.mean(rouge_scores)
    }


In [106]:
%pip install --upgrade --force-reinstall accelerate>=0.26.0


^C
Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.5.1 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [107]:
import accelerate

In [3]:
training_args = TrainingArguments(
    output_dir="./qwen25-0.5b-prefix-tuning",
    learning_rate=1e-3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
    report_to="wandb",
    run_name="qwen25-0.5b-prefix-tuning",
)

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train,
    eval_dataset=small_eval,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
# Train the model
trainer.train()

In [ ]:
# Evaluate prefix-tuned model
eval_results = trainer.evaluate()
print("Prefix-Tuned Model Scores:", eval_results)

# Log to W&B
wandb.log({
    "prefix_bleu": eval_results["eval_bleu"],
    "prefix_rougeL": eval_results["eval_rougeL"]
})


In [ ]:
# Compare with base model
print("\nComparison:")
print(f"Base Model - BLEU: {base_bleu_avg:.4f}, ROUGE-L: {base_rouge_avg:.4f}")
print(f"Prefix-Tuned - BLEU: {eval_results['eval_bleu']:.4f}, ROUGE-L: {eval_results['eval_rougeL']:.4f}")

In [ ]:
# Push model to Hugging Face Hub
trainer.push_to_hub("qwen25-0.5b-prefix-tuning-question-generation")

In [ ]:
# Finish W&B run
wandb.finish()